
# Análisis Financiero con Bandas de Bollinger y Singleton

Este notebook muestra cómo aplicar el patrón Singleton para manejar la conexión a una base de datos SQLite y realizar un análisis financiero con bandas de Bollinger usando datos sintéticos.


In [7]:

import random
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


In [8]:

class ConexionDB:
    _instancia = None

    def __init__(self, nombre_bd='empresa.db'):
        if ConexionDB._instancia is not None:
            raise Exception("Esta clase es un Singleton. Usa get_instancia().")
        self.nombre_bd = nombre_bd
        self.conexion = sqlite3.connect(self.nombre_bd)
        ConexionDB._instancia = self

    @staticmethod
    def get_instancia(nombre_bd='empresa.db'):
        if ConexionDB._instancia is None:
            ConexionDB(nombre_bd)
        return ConexionDB._instancia

    def obtener_conexion(self):
        return self.conexion

    def cerrar(self):
        if self.conexion:
            self.conexion.close()
            ConexionDB._instancia = None


In [9]:

def inicializar_base():
    conexion = ConexionDB.get_instancia().obtener_conexion()
    cursor = conexion.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS empresas (
        id INTEGER PRIMARY KEY,
        ticker TEXT NOT NULL,
        nombre TEXT NOT NULL
    )''')

    empresas = [
        (1, 'AAPL', 'Apple Inc.'),
        (2, 'GOOGL', 'Alphabet Inc.'),
        (3, 'MSFT', 'Microsoft Corporation'),
        (4, 'AMZN', 'Amazon.com Inc.'),
        (5, 'TSLA', 'Tesla Inc.'),
        (6, 'META', 'Meta Platforms Inc.'),
        (7, 'NVDA', 'NVIDIA Corporation'),
        (8, 'NFLX', 'Netflix Inc.'),
        (9, 'ADBE', 'Adobe Inc.'),
        (10, 'ORCL', 'Oracle Corporation')
    ]

    cursor.executemany('''
    INSERT OR IGNORE INTO empresas (id, ticker, nombre)
    VALUES (?, ?, ?)
    ''', empresas)

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS SerieTemporal (
        id INTEGER PRIMARY KEY,
        empresa_id INTEGER,
        valor REAL,
        fecha TEXT,
        FOREIGN KEY (empresa_id) REFERENCES empresas(id)
    )''')

    fecha_inicio = datetime(2023, 1, 1)
    entradas = 100
    datos_temporales = []

    for empresa in empresas:
        empresa_id = empresa[0]
        for i in range(entradas):
            fecha = fecha_inicio + timedelta(days=i)
            valor = round(random.uniform(100, 500), 2)
            datos_temporales.append((empresa_id, valor, fecha.strftime('%Y-%m-%d')))

    cursor.executemany('''
    INSERT INTO SerieTemporal (empresa_id, valor, fecha)
    VALUES (?, ?, ?)
    ''', datos_temporales)

    conexion.commit()


In [10]:

class Empresa:
    def __init__(self, empresa_id, ticker, nombre):
        self.empresa_id = empresa_id
        self.ticker = ticker
        self.nombre = nombre
        self.serie_temporal = None
        self.media_movil = None
        self.banda_superior = None
        self.banda_inferior = None
        self.calificacion = None

    def cargar_serie_temporal(self, conexion):
        query = '''
        SELECT fecha, valor
        FROM SerieTemporal
        WHERE empresa_id = ?
        ORDER BY fecha
        '''
        self.serie_temporal = pd.read_sql_query(query, conexion, params=(self.empresa_id,))
        self.serie_temporal['fecha'] = pd.to_datetime(self.serie_temporal['fecha'])

    def calcular_bandas_bollinger(self, ventana=20, ancho=2):
        media = self.serie_temporal['valor'].rolling(ventana).mean()
        desviacion = self.serie_temporal['valor'].rolling(ventana).std()
        self.media_movil = media
        self.banda_superior = media + (ancho * desviacion)
        self.banda_inferior = media - (ancho * desviacion)

    def asignar_calificacion(self):
        ultimo_valor = self.serie_temporal['valor'].iloc[-1]
        if ultimo_valor > self.banda_superior.iloc[-1]:
            self.calificacion = 'A'
        elif ultimo_valor < self.banda_inferior.iloc[-1]:
            self.calificacion = 'C'
        else:
            self.calificacion = 'B'

    def mostrar(self):
        print(f'\nEmpresa: {self.nombre} ({self.ticker})')
        print(f'Calificación: {self.calificacion}')
        print('\nÚltimos datos de la serie temporal:')
        print(self.serie_temporal.tail())
        print('\nMedia Móvil:')
        print(self.media_movil.tail())
        print('\nBanda Superior:')
        print(self.banda_superior.tail())
        print('\nBanda Inferior:')
        print(self.banda_inferior.tail())


In [11]:

def obtener_empresa_por_ticker_o_id(identificador, conexion):
    cursor = conexion.cursor()
    if isinstance(identificador, int):
        query = 'SELECT id, ticker, nombre FROM empresas WHERE id = ?'
    else:
        query = 'SELECT id, ticker, nombre FROM empresas WHERE ticker = ?'

    cursor.execute(query, (identificador,))
    fila = cursor.fetchone()
    if fila:
        return Empresa(fila[0], fila[1], fila[2])
    return None


In [12]:

inicializar_base()

instancia_db = ConexionDB.get_instancia()
conexion = instancia_db.obtener_conexion()

empresa = obtener_empresa_por_ticker_o_id('GOOGL', conexion)
if empresa:
    empresa.cargar_serie_temporal(conexion)
    empresa.calcular_bandas_bollinger()
    empresa.asignar_calificacion()
    empresa.mostrar()

instancia_db.cerrar()



Empresa: Alphabet Inc. (GOOGL)
Calificación: B

Últimos datos de la serie temporal:
        fecha   valor
95 2023-04-06  342.20
96 2023-04-07  301.04
97 2023-04-08  294.89
98 2023-04-09  399.50
99 2023-04-10  107.33

Media Móvil:
95    283.6750
96    282.2255
97    291.9070
98    292.2340
99    290.6065
Name: valor, dtype: float64

Banda Superior:
95    543.167290
96    540.950274
97    536.208438
98    537.121760
99    540.146010
Name: valor, dtype: float64

Banda Inferior:
95    24.182710
96    23.500726
97    47.605562
98    47.346240
99    41.066990
Name: valor, dtype: float64
